In [1]:
from __future__ import print_function, division
from builtins import range

In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [3]:
train = pd.read_csv('large_files/r8-train-all-terms.txt', header=None, sep='\t')
test = pd.read_csv('large_files/r8-test-all-terms.txt', header=None, sep='\t')
train.columns = ['label', 'content']
test.columns = ['label', 'content']


In [4]:
print(train.content[1])
print("\t",train.label[1])

computer terminal systems cpml completes sale computer terminal systems inc said it has completed the sale of shares of its common stock and warrants to acquire an additional one mln shares to sedio n v of lugano switzerland for dlrs the company said the warrants are exercisable for five years at a purchase price of dlrs per share computer terminal said sedio also has the right to buy additional shares and increase its total holdings up to pct of the computer terminal s outstanding common stock under certain circumstances involving change of control at the company the company said if the conditions occur the warrants would be exercisable at a price equal to pct of its common stock s market price at the time not to exceed dlrs per share computer terminal also said it sold the technolgy rights to its dot matrix impact technology including any future improvements to woodco inc of houston tex for dlrs but it said it would continue to be the exclusive worldwide licensee of the technology fo

In [5]:
class GloveVectorizer:
    def __init__(self):
        print('loading word vectors...')
        word2vec = {}
        embedding = []
        idx2word = []
        with open('large_files/glove.6b/glove.6b.50d.txt', encoding="utf8") as f:
            for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:], dtype='float32')
                word2vec[word] = vec
                embedding.append(vec)
                idx2word.append(word)
        print('Found %s word vectors.' % len(word2vec))
        
        self.word2vec = word2vec
        self.embedding = np.array(embedding)
        self.word2idx = {v:k for k,v in enumerate(idx2word)}
        self.V, self.D = self.embedding.shape
        
    def fit(self, data):
        pass
    
    def transform(self, data):
        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            for word in tokens:
                if word in self.word2vec:
                    vec = self.word2vec[word]
                    vecs.append(vec)
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount +=1
            n += 1
        print("Number of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X
    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [6]:
class Woord2VecVectorizer:
    def __init__(self):
        print('Loading in word vectors...')
        self.word_vectors = KeyedVectors.load_word2vec_format('large_files/GoogleNews-vectors-negative300.bin', binary=True)
        print("Finished loading in word vectors")
        
    def fit(self, data):
        pass
    
    def transform(self, data):
        v = self.word_vectors.get_vector('king')
        self.D = v.shape[0]
        
        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.split()
            vecs = []
            m = 0
            for word in tokens:
                try:
                    vec = self.word_vectors.get_vector(word)
                    vecs.append(vec)
                    m += 1
                except KeyError:
                    pass
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount += 1
            n += 1
        print("Number of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X
    
    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [7]:
vectorizer = GloveVectorizer()
Xtrain = vectorizer.fit_transform(train.content)
Ytrain = train.label

Xtest = vectorizer.fit_transform(train.content)
Ytest = train.label

loading word vectors...
Found 400000 word vectors.
Number of samples with no words found: 0 / 5485
Number of samples with no words found: 0 / 5485


In [8]:
model = RandomForestClassifier(n_estimators=200)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

train score: 0.9992707383773929
test score: 0.9992707383773929


In [9]:
print(Ytest.unique())

['earn' 'acq' 'trade' 'ship' 'grain' 'crude' 'interest' 'money-fx']


In [10]:
print(Xtrain[1])

[ 0.42225829  0.07437593  0.26020068  0.23492537  0.29256091  0.08535112
 -0.35343158 -0.39948615  0.09618001  0.00716475  0.16270347  0.08050574
 -0.25246552 -0.2461143   0.31502548  0.28226331 -0.16852508  0.01078145
 -0.33009204 -0.39175874  0.45302069 -0.11710789 -0.06598587 -0.1905102
 -0.26123828 -1.29367924 -0.14937761 -0.11885012  0.0887498  -0.05608343
  3.09159064  0.10806133 -0.07475229 -0.14376929  0.14825106 -0.25082457
 -0.003683    0.04742439  0.10568412 -0.13436335 -0.03710696  0.01067496
  0.2158376   0.16028534 -0.23386878  0.07255986 -0.14271812  0.17860013
 -0.03726814  0.00975073]
